Hack Environment Set Up
========================
Install the necessary client libraries in order to demo running arbitrary code on AI compute platform. Those steps are not needed once productionized. 

In [0]:
%sh git clone -b prototype "https://github.com/ygong1/llm-foundry.git" ~/llm-foundry

In [0]:
%sh cd ~/llm-foundry && pip install .

In [0]:
dbutils.library.restartPython()

Load the Data and Perform Data Exploration before Training
----------------------------------------------------------

This [data processing notebook](https://e2-dogfood.staging.cloud.databricks.com/?o=6051921418418893#notebook/3021853473816147/command/4183847697906984) showed the details how to load CIFAR dataset into UC table and UC volumn. 

At the moment, StreamDataset can only process MDS formate that's differnt from the format that spark can read. We are saving the dataset in 2 copies
1. `main.yu_gong.cifar10_train` to be loaded by spark natively to show how to do data manipulation
2. `/Volumes/main/yu_gong/cifar10/train/` to be loaded by StreamingDataset

In the demo, we hack to convert `main.yu_gong.cifar10_train` to `/Volumes/main/yu_gong/cifar10/train/` to show the idea that eventually they are going to be the same dataset, at least from user perspective


In [0]:
# check the distribution of label
df = spark.read.table('main.yu_gong.cifar10_train') 
distribution = df.groupBy("label").count().orderBy("count", ascending=False)
display(distribution)

Submit Run Remotely On a Cluster
-------------------------------

Wrap up the config and submit the remote run.


In [0]:
from ygong.mosaic import ScalingConfig
from ygong.mosaic import submit
from ygong.mosaic import TrainingConfig
from ygong.mosaic.wsfs import WSFSIntegration

parameters = { 
     "name": "custom-train-demo", 
     "seed": 42, 
     "device_train_microbatch_size": 8,
     "data": "main.yu_gong.cifar10_train",
     "loggers": {
        "mlflow": {
            "tracking_uri": "databricks",
            "synchronous": False,
            "log_system_metrics": True
        }
     }
}
custom_code_repo_dir = "/Workspace/Users/yu.gong@databricks.com/.ide/custom-train-demo-4bf5c137"
# TODO(shitao): add a prioirty field with good default. We can show it in demo
config = TrainingConfig(
    name="custom-train-demo", 
    entry_point=f'{custom_code_repo_dir}/src/train.py', 
    parameters=parameters)

scalingConfig = ScalingConfig(gpusNum=8, poolName="staging-aws-us-east-1-mlserv1-gentrain1")

# DEMO NOTE:
# This is temporary hack to mock the behavior of mounting workspace filesystem to the remote training
# nodes. Once the workspace filesystem fusion is integrated with netphos and dblet. We will get this for free.
wsfs = WSFSIntegration(wsfs_path=custom_code_repo_dir)

run = submit(config, scalingConfig, wait_job_to_finish=True, debug=True, wsfs=wsfs)

In [0]:
from mcli import RunStatus
if run.status == RunStatus.COMPLETED:
  print("Succeeded!")
else:
  raise Exception("Pretaining job faile")


